<a href="https://colab.research.google.com/github/harshitabhambhani/ML-DL-models/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!pip install tensorflow


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load stock data
symbol = "MSFT"  # Change this to the stock symbol you are interested in
start_date = "2010-01-01"
end_date = "2022-01-01"

data = yf.download(symbol, start=start_date, end=end_date)

# Use the closing prices for prediction
df = data[['Close']]


[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Create sequences for training the LSTM model
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]
        X.append(seq)
        y.append(label)
    return np.array(X), np.array(y)

# Choose sequence length (e.g., 20 days)
sequence_length = 20

# Create sequences
X, y = create_sequences(df_scaled, sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50
75/75 [==============================] - 8s 47ms/step - loss: 0.0078
Epoch 2/50
75/75 [==============================] - 1s 19ms/step - loss: 1.5253e-04
Epoch 3/50
75/75 [==============================] - 1s 18ms/step - loss: 1.5836e-04
Epoch 4/50
75/75 [==============================] - 1s 19ms/step - loss: 1.5392e-04
Epoch 5/50
75/75 [==============================] - 1s 19ms/step - loss: 1.4832e-04
Epoch 6/50
75/75 [==============================] - 1s 18ms/step - loss: 1.5555e-04
Epoch 7/50
75/75 [==============================] - 1s 16ms/step - loss: 1.4489e-04
Epoch 8/50
75/75 [==============================] - 2s 29ms/step - loss: 1.4238e-04
Epoch 9/50
75/75 [==============================] - 2s 22ms/step - loss: 1.2750e-04
Epoch 10/50
75/75 [==============================] - 1s 19ms/step - loss: 1.4416e-04
Epoch 11/50
75/75 [==============================] - 1s 18ms/step - loss: 2.1710e-04
Epoch 12/50
75/75 [==============================] - 1s 18ms/step - loss: 1.45

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
print("Root Mean Squared Error:", rmse)


19/19 [==============================] - 2s 11ms/step
Root Mean Squared Error: 2.3576012101328776
